In [ ]:
!pip install -q requests dnspython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 24.2 MB/s eta 0:00:00


In [ ]:
import time
import requests
import dns.resolver
import pandas as pd


In [ ]:
def measure_http_latency(url, trials=3):
    times = []
    for _ in range(trials):
        start = time.time()
        try:
            requests.get(url, timeout=5)
            times.append(time.time() - start)
        except:
            times.append(None)
    return times


In [ ]:
def measure_dns_latency(domain, trials=3):
    times = []
    resolver = dns.resolver.Resolver()
    for _ in range(trials):
        start = time.time()
        try:
            resolver.resolve(domain)
            times.append(time.time() - start)
        except:
            times.append(None)
    return times


In [ ]:
targets = {
    "Google": "https://www.google.com",
    "Cloudflare": "https://www.cloudflare.com",
    "Wikipedia": "https://www.wikipedia.org"
}

results = []

for name, url in targets.items():
    http_times = measure_http_latency(url)
    dns_times = measure_dns_latency(url.replace("https://www.", ""))

    for i in range(len(http_times)):
        results.append({
            "service": name,
            "http_latency_sec": http_times[i],
            "dns_latency_sec": dns_times[i]
        })

df = pd.DataFrame(results)
df


,service,http_latency_sec,dns_latency_sec
0,Google,0.051594,0.003625
1,Google,0.042850,0.001329
2,Google,0.051482,0.001146
3,Cloudflare,0.427465,0.006355
4,Cloudflare,0.477214,0.001526
5,Cloudflare,0.430985,0.001221
6,Wikipedia,0.011421,0.002972
7,Wikipedia,0.011691,0.002630
8,Wikipedia,0.011318,0.002297


In [ ]:
df.groupby("service").mean()


,http_latency_sec,dns_latency_sec
service,,
Cloudflare,0.445222,0.003034
Google,0.048642,0.002033
Wikipedia,0.011476,0.002633
